<h2 align="center">Atliq Hotels Analysis Project<h2>

In [143]:
import pandas as pd

In [73]:
# Reading csv files
df_bookings = pd.read_csv('datasets/fact_bookings.csv')
df_date = pd.read_csv('datasets/dim_date.csv')
df_hotels = pd.read_csv('datasets/dim_hotels.csv')
df_rooms = pd.read_csv('datasets/dim_rooms.csv')
df_agg_bookings = pd.read_csv('datasets/fact_aggregated_bookings.csv')

***
<h2 align='center'> Data Exploration <h1>

***

In [74]:
# Total rows and columns in bookings
df_bookings.shape

(134590, 12)

In [56]:
df_agg_bookings.shape

(9200, 5)

In [8]:
df_agg_bookings.head() # Bookings info per property

,property_id,check_in_date,room_category,successful_bookings,capacity
0,16559,1-May-22,RT1,25,30.0
1,19562,1-May-22,RT1,28,30.0
2,19563,1-May-22,RT1,23,30.0
3,17558,1-May-22,RT1,30,19.0
4,16558,1-May-22,RT1,18,19.0


In [10]:
df_bookings.head() # Bookings real-time data

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
0,May012216558RT11,16558,27-04-22,1/5/2022,2/5/2022,-3.0,RT1,direct online,1.0,Checked Out,10010,10010
1,May012216558RT12,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640
2,May012216558RT13,16558,28-04-22,1/5/2022,4/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100000,9100
3,May012216558RT14,16558,28-04-22,1/5/2022,2/5/2022,-2.0,RT1,others,NaN,Cancelled,9100,3640
4,May012216558RT15,16558,27-04-22,1/5/2022,2/5/2022,4.0,RT1,direct online,5.0,Checked Out,10920,10920


In [14]:
# DISTINCT ROOM class
df_rooms.head()

,room_id,room_class
0,RT1,Standard
1,RT2,Elite
2,RT3,Premium
3,RT4,Presidential


In [18]:
df_rooms.room_class.unique()

array(['Standard', 'Elite', 'Premium', 'Presidential'], dtype=object)

#### Bookings recorded per platform

In [75]:
df_bookings.booking_platform.value_counts()

booking_platform
others            55066
makeyourtrip      26898
logtrip           14756
direct online     13379
tripster           9630
journey            8106
direct offline     6755
Name: count, dtype: int64

**Find out unique property ids in aggregate bookings dataset**

In [272]:
df_agg_bookings.property_id.unique()

array([16559, 19562, 19563, 17558, 16558, 17560, 19558, 19560, 17561,
       16560, 16561, 16562, 16563, 17559, 17562, 17563, 18558, 18559,
       18561, 18562, 18563, 19559, 19561, 17564, 18560], dtype=int64)

**Find out total bookings per property_id**

In [274]:
df_agg_bookings.groupby('property_id')['successful_bookings'].sum()

property_id
16558    3153
16559    7338
16560    4693
16561    4418
16562    4820
16563    7211
17558    5053
17559    6142
17560    6013
17561    5183
17562    3424
17563    6337
17564    3982
18558    4475
18559    5256
18560    6638
18561    6458
18562    7333
18563    4737
19558    4400
19559    4729
19560    6079
19561    5736
19562    5812
19563    5413
Name: successful_bookings, dtype: int64

**Find out days on which bookings are greater than capacity**

In [277]:
df_agg_bookings[df_agg_bookings.successful_bookings>df_agg_bookings.capacity]

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
3,17558,1-May-22,RT1,30,19.0,157.89
12,16563,1-May-22,RT1,100,41.0,243.90
4136,19558,11-Jun-22,RT2,50,39.0,128.21
6209,19560,2-Jul-22,RT1,123,26.0,473.08
8522,19559,25-Jul-22,RT1,35,24.0,145.83
9194,18563,31-Jul-22,RT4,20,18.0,111.11


**Find out properties that have highest capacity**

In [278]:
df_agg_bookings.capacity.max()

41.0

In [280]:
df_agg_bookings[df_agg_bookings.capacity == df_agg_bookings.capacity.max()]

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
12,16563,1-May-22,RT1,100,41.0,243.9


##### Quick Stats

In [76]:
df_bookings.describe().round(2)

,property_id,no_guests,ratings_given,revenue_generated,revenue_realized
count,134590.00,134587.00,56683.00,134590.00,134590.00
mean,18061.11,2.04,3.62,15378.05,12696.12
std,1093.06,1.03,1.24,93036.04,6928.11
min,16558.00,-17.00,1.00,6500.00,2600.00
25%,17558.00,1.00,3.00,9900.00,7600.00
50%,17564.00,2.00,4.00,13500.00,11700.00
75%,18563.00,2.00,5.00,18000.00,15300.00
max,19563.00,6.00,5.00,28560000.00,45220.00


***
<h2 align='center'> Data Cleaning <h2>

***

***In aggregate bookings find columns that have null values. Fill these null values with whatever you think is the appropriate subtitute (possible ways is to use mean or median)***

In [282]:
#columns that have null values

df_agg_bookings.isnull().sum()

property_id            0
check_in_date          0
room_category          0
successful_bookings    0
capacity               0
occ_pct                0
dtype: int64

In [ ]:
'''
Data Anomalies detected!
1. Minimum no of guests are -17, which can't be possible. we've to fix this
2. Max Revenue Generated is 28560000.00 
'''


In [77]:
# Find no of records which have negative values for guest size
df_bookings[df_bookings.no_guests<0]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
0,May012216558RT11,16558,27-04-22,1/5/2022,2/5/2022,-3.0,RT1,direct online,1.0,Checked Out,10010,10010
3,May012216558RT14,16558,28-04-22,1/5/2022,2/5/2022,-2.0,RT1,others,NaN,Cancelled,9100,3640
17924,May122218559RT44,18559,12/5/2022,12/5/2022,14-05-22,-10.0,RT4,direct online,NaN,No Show,20900,20900
18020,May122218561RT22,18561,8/5/2022,12/5/2022,14-05-22,-12.0,RT2,makeyourtrip,NaN,Cancelled,9000,3600
18119,May122218562RT311,18562,5/5/2022,12/5/2022,17-05-22,-6.0,RT3,direct offline,5.0,Checked Out,16800,16800
18121,May122218562RT313,18562,10/5/2022,12/5/2022,17-05-22,-4.0,RT3,direct online,NaN,Cancelled,14400,5760
56715,Jun082218562RT12,18562,5/6/2022,8/6/2022,13-06-22,-17.0,RT1,others,NaN,Checked Out,6500,6500
119765,Jul202219560RT220,19560,19-07-22,20-07-22,22-07-22,-1.0,RT2,others,NaN,Checked Out,13500,13500
134586,Jul312217564RT47,17564,30-07-22,31-07-22,1/8/2022,-4.0,RT4,logtrip,2.0,Checked Out,38760,38760


In [78]:
# IGNORE THESE RECORDS AS THEY ARE NOT SIGNIFICANT IN NUMBERS, WON'T AFFECT OUR ANALYSIS
df_bookings = df_bookings[df_bookings.no_guests>0]


#### Outlier detection for Revenue Generated column

In [80]:
df_bookings.revenue_generated.min(), df_bookings.revenue_generated.max()

(6500, 28560000)

##### Approach : STD Deviation

In [81]:
avg, std = df_bookings.revenue_generated.mean().round(2), df_bookings.revenue_generated.std().round(2)
avg,std

(15378.04, 93040.15)

In [82]:
lower_limit = avg - 3*std
higher_limit = avg + 3*std

lower_limit, higher_limit

(-263742.41, 294498.48999999993)

In [43]:
df_bookings[df_bookings.revenue_generated<lower_limit]
# No records below 3 std point

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized


In [83]:
df_bookings[df_bookings.revenue_generated>higher_limit]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
2,May012216558RT13,16558,28-04-22,1/5/2022,4/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100000,9100
111,May012216559RT32,16559,29-04-22,1/5/2022,2/5/2022,6.0,RT3,direct online,NaN,Checked Out,28560000,28560
315,May012216562RT22,16562,28-04-22,1/5/2022,4/5/2022,2.0,RT2,direct offline,3.0,Checked Out,12600000,12600
562,May012217559RT118,17559,26-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,2000000,4420
129176,Jul282216562RT26,16562,21-07-22,28-07-22,29-07-22,2.0,RT2,direct online,3.0,Checked Out,10000000,12600


In [84]:
# These records are outliers & contribute to the right-skewed distribution of the data. So we'll remove them
df_bookings = df_bookings[df_bookings.revenue_generated<higher_limit]
df_bookings.tail()

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
134584,Jul312217564RT45,17564,30-07-22,31-07-22,1/8/2022,2.0,RT4,others,2.0,Checked Out,32300,32300
134585,Jul312217564RT46,17564,29-07-22,31-07-22,3/8/2022,1.0,RT4,makeyourtrip,2.0,Checked Out,32300,32300
134587,Jul312217564RT48,17564,30-07-22,31-07-22,2/8/2022,1.0,RT4,tripster,NaN,Cancelled,32300,12920
134588,Jul312217564RT49,17564,29-07-22,31-07-22,1/8/2022,2.0,RT4,logtrip,2.0,Checked Out,32300,32300
134589,Jul312217564RT410,17564,31-07-22,31-07-22,1/8/2022,2.0,RT4,makeyourtrip,NaN,Cancelled,32300,12920


In [85]:
df_bookings.shape

(134573, 12)

In [86]:
# CHECK FOR REVENUE_REALIZED COLUMN

In [87]:
df_bookings.revenue_realized.min(), df_bookings.revenue_realized.max()

(2600, 45220)

In [89]:
avg, std = df_bookings.revenue_realized.mean(), df_bookings.revenue_realized.std()

avg,std

(12695.983585117372, 6927.791692242814)

In [92]:
lower_limit= avg - 3*std
higher_limit = avg + 3*std

lower_limit, higher_limit

(-8087.391491611072, 33479.358661845814)

In [93]:
df_bookings[df_bookings.revenue_realized>higher_limit]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
137,May012216559RT41,16559,27-04-22,1/5/2022,7/5/2022,4.0,RT4,others,NaN,Checked Out,38760,38760
139,May012216559RT43,16559,1/5/2022,1/5/2022,2/5/2022,6.0,RT4,tripster,3.0,Checked Out,45220,45220
143,May012216559RT47,16559,28-04-22,1/5/2022,3/5/2022,3.0,RT4,others,5.0,Checked Out,35530,35530
149,May012216559RT413,16559,24-04-22,1/5/2022,7/5/2022,5.0,RT4,logtrip,NaN,Checked Out,41990,41990
222,May012216560RT45,16560,30-04-22,1/5/2022,3/5/2022,5.0,RT4,others,3.0,Checked Out,34580,34580
...,...,...,...,...,...,...,...,...,...,...,...,...
134328,Jul312219560RT49,19560,31-07-22,31-07-22,2/8/2022,6.0,RT4,direct online,5.0,Checked Out,39900,39900
134331,Jul312219560RT412,19560,31-07-22,31-07-22,1/8/2022,6.0,RT4,others,2.0,Checked Out,39900,39900
134467,Jul312219562RT45,19562,28-07-22,31-07-22,1/8/2022,6.0,RT4,makeyourtrip,4.0,Checked Out,39900,39900
134474,Jul312219562RT412,19562,25-07-22,31-07-22,6/8/2022,5.0,RT4,direct offline,5.0,Checked Out,37050,37050


In [94]:
df_rooms[df_rooms["room_id"] == 'RT4']

,room_id,room_class
3,RT4,Presidential


In [ ]:
'''
ANALYSIS!
We noticed that for the revenue realised column, 3STD or higher limit for an outlier is 33479.358661845814 
and we have records that exceed the higher limit but all of them belong to the same room category RT4 which
is a presidential suit and they are expected to be that much higher than other room types. 
So we won't consider them as outliers.

'''

***
<h2 align='center'> Data Transformation <h2>

***

In [99]:
df_agg_bookings.head()

,property_id,check_in_date,room_category,successful_bookings,capacity
0,16559,1-May-22,RT1,25,30.0
1,19562,1-May-22,RT1,28,30.0
2,19563,1-May-22,RT1,23,30.0
3,17558,1-May-22,RT1,30,19.0
4,16558,1-May-22,RT1,18,19.0


In [106]:
# Calculating Occupancy Percentage = successful bookings/ capacity

df_agg_bookings["occ_pct"] = df_agg_bookings["successful_bookings"]/df_agg_bookings["capacity"]
df_agg_bookings.head()

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
0,16559,1-May-22,RT1,25,30.0,0.833333
1,19562,1-May-22,RT1,28,30.0,0.933333
2,19563,1-May-22,RT1,23,30.0,0.766667
3,17558,1-May-22,RT1,30,19.0,1.578947
4,16558,1-May-22,RT1,18,19.0,0.947368


In [107]:
# Converting values of calculated column occ_pct into % using apply and lambda function

df_agg_bookings["occ_pct"] = df_agg_bookings["occ_pct"].apply(lambda x: round(x*100, 2))
df_agg_bookings.head()

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
0,16559,1-May-22,RT1,25,30.0,83.33
1,19562,1-May-22,RT1,28,30.0,93.33
2,19563,1-May-22,RT1,23,30.0,76.67
3,17558,1-May-22,RT1,30,19.0,157.89
4,16558,1-May-22,RT1,18,19.0,94.74


***
<h2 align="center"> Ad-Hoc requests <h2>
   
***

### 1. What is an average occupancy rate in each of the room categories?

In [109]:
df_rooms.head(2)

,room_id,room_class
0,RT1,Standard
1,RT2,Elite


In [110]:
df_agg_bookings.head(2)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct
0,16559,1-May-22,RT1,25,30.0,83.33
1,19562,1-May-22,RT1,28,30.0,93.33


In [111]:
df_agg_bookings.groupby("room_category")['occ_pct'].mean()

room_category
RT1    58.224247
RT2    58.040278
RT3    58.028213
RT4    59.300461
Name: occ_pct, dtype: float64

In [114]:
df_prop_agg_bookings = pd.merge(df_agg_bookings, df_rooms, left_on='room_category', right_on = 'room_id')
df_prop_agg_bookings.head(2)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_id,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,RT1,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,RT1,Standard


In [118]:
# Removed duplicated column room_id
df_prop_agg_bookings.drop('room_id', axis=1, inplace=True)

df_prop_agg_bookings.head(2)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,Standard


In [123]:
df_prop_agg_bookings.groupby("room_class")['occ_pct'].mean().round(2).sort_values(ascending=False)

room_class
Presidential    59.30
Standard        58.22
Elite           58.04
Premium         58.03
Name: occ_pct, dtype: float64

Presidential suit has max avg occupancy rate 


In [131]:
df_prop_agg_bookings[df_prop_agg_bookings.room_class=='Presidential']['occ_pct'].mean().round(2)

59.3

### 2. Print average occupancy rate per city

In [135]:
df_hotels.head(3)

,property_id,property_name,category,city
0,16558,Atliq Grands,Luxury,Delhi
1,16559,Atliq Exotica,Luxury,Mumbai
2,16560,Atliq City,Business,Delhi


In [137]:
df_prop_agg_bookings.head(3)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class
0,16559,1-May-22,RT1,25,30.0,83.33,Standard
1,19562,1-May-22,RT1,28,30.0,93.33,Standard
2,19563,1-May-22,RT1,23,30.0,76.67,Standard


In [139]:
df_bookings_city= pd.merge(df_prop_agg_bookings, df_hotels, on='property_id')
df_bookings_city.head(2)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class,property_name,category,city
0,16559,1-May-22,RT1,25,30.0,83.33,Standard,Atliq Exotica,Luxury,Mumbai
1,16559,2-May-22,RT1,20,30.0,66.67,Standard,Atliq Exotica,Luxury,Mumbai


In [148]:
df_bookings_city.groupby('city')["occ_pct"].mean().round(2).sort_values()

city
Bangalore    56.59
Mumbai       57.94
Hyderabad    58.14
Delhi        61.61
Name: occ_pct, dtype: float64

Delhi has Maximum avg Occupancy 

#### 3. When was the occupancy better? Weekday or Weekend?

In [151]:
df_date.head(2)

,date,mmm yy,week no,day_type
0,01-May-22,May 22,W 19,weekend
1,02-May-22,May 22,W 19,weekeday


In [153]:
df_bookings_city = pd.merge(df_bookings_city, df_date, left_on='check_in_date', right_on='date')
df_bookings_city.head(2)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class,property_name,category,city,date_x,mmm yy_x,week no_x,day_type_x,date_y,mmm yy_y,week no_y,day_type_y
0,16559,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Exotica,Luxury,Mumbai,10-May-22,May 22,W 20,weekeday,10-May-22,May 22,W 20,weekeday
1,16559,10-May-22,RT2,25,41.0,60.98,Elite,Atliq Exotica,Luxury,Mumbai,10-May-22,May 22,W 20,weekeday,10-May-22,May 22,W 20,weekeday


In [155]:
df_bookings_city.drop('date_x', axis = 1, inplace=True)
df_bookings_city.drop('date_y', axis = 1, inplace=True)
df_bookings_city.drop('mmm yy_y', axis = 1, inplace=True)
df_bookings_city.drop('week no_y', axis = 1, inplace=True)
df_bookings_city.drop('day_type_y', axis = 1, inplace=True)

# Dropped extra columns that were added after running the above code twice
df_bookings_city

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class,property_name,category,city,mmm yy_x,week no_x,day_type_x
0,16559,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
1,16559,10-May-22,RT2,25,41.0,60.98,Elite,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
2,16559,10-May-22,RT3,20,32.0,62.50,Premium,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
3,16559,10-May-22,RT4,13,18.0,72.22,Presidential,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
4,19562,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Bay,Luxury,Bangalore,May 22,W 20,weekeday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,17564,31-Jul-22,RT4,10,17.0,58.82,Presidential,Atliq Seasons,Business,Mumbai,Jul 22,W 32,weekend
6496,18560,31-Jul-22,RT1,22,30.0,73.33,Standard,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend
6497,18560,31-Jul-22,RT2,34,40.0,85.00,Elite,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend
6498,18560,31-Jul-22,RT3,17,24.0,70.83,Premium,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend


In [157]:
df_bookings_city.groupby('day_type_x')['occ_pct'].mean().round(2)

day_type_x
weekeday    50.90
weekend     72.39
Name: occ_pct, dtype: float64

Weekends has better occupancy than weekdays as expeceted

#### 4: In the month of June, what is the occupancy for different cities

In [168]:
# create june dataframe for analysis
df_june22_bookings = df_bookings_city[df_bookings_city['mmm yy_x'] == 'Jun 22']

df_june22_bookings.tail(3)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ_pct,room_class,property_name,category,city,mmm yy_x,week no_x,day_type_x
4297,18560,30-Jun-22,RT2,24,40.0,60.00,Elite,Atliq City,Business,Hyderabad,Jun 22,W 27,weekeday
4298,18560,30-Jun-22,RT3,14,24.0,58.33,Premium,Atliq City,Business,Hyderabad,Jun 22,W 27,weekeday
4299,18560,30-Jun-22,RT4,8,15.0,53.33,Presidential,Atliq City,Business,Hyderabad,Jun 22,W 27,weekeday


In [171]:
df_june22_bookings.groupby('city')['occ_pct'].mean().round(2).sort_values(ascending=False)

city
Delhi        62.47
Hyderabad    58.46
Mumbai       58.38
Bangalore    56.58
Name: occ_pct, dtype: float64

#### 5: We got new data for the month of august. Append that to existing data

In [175]:
df_aug22_data = pd.read_csv('datasets/new_data_august.csv')

df_aug22_data

,property_id,property_name,category,city,room_category,room_class,check_in_date,mmm yy,week no,day_type,successful_bookings,capacity,occ%
0,16559,Atliq Exotica,Luxury,Mumbai,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,30,30,100.00
1,19562,Atliq Bay,Luxury,Bangalore,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,21,30,70.00
2,19563,Atliq Palace,Business,Bangalore,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,23,30,76.67
3,19558,Atliq Grands,Luxury,Bangalore,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,30,40,75.00
4,19560,Atliq City,Business,Bangalore,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,20,26,76.92
5,17561,Atliq Blu,Luxury,Mumbai,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,18,26,69.23
6,17564,Atliq Seasons,Business,Mumbai,RT1,Standard,01-Aug-22,Aug-22,W 32,weekeday,10,16,62.50


**We've to change the header's name of df_bookings_city as for column concatenation, make sure columns are same**

In [178]:
df_bookings_city =df_bookings_city.rename(columns=
                                         {'mmm yy_x': 'mmm yy',
                                           'week no_x': 'week no',	
                                           'day_type_x': 'day_type' 
                                             
                                         })


In [186]:
df_bookings_city = df_bookings_city.rename(columns={
                                    'occ_pct': 'occ %'
})

df_bookings_city

,property_id,check_in_date,room_category,successful_bookings,capacity,occ %,room_class,property_name,category,city,mmm yy,week no,day_type
0,16559,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
1,16559,10-May-22,RT2,25,41.0,60.98,Elite,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
2,16559,10-May-22,RT3,20,32.0,62.50,Premium,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
3,16559,10-May-22,RT4,13,18.0,72.22,Presidential,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
4,19562,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Bay,Luxury,Bangalore,May 22,W 20,weekeday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,17564,31-Jul-22,RT4,10,17.0,58.82,Presidential,Atliq Seasons,Business,Mumbai,Jul 22,W 32,weekend
6496,18560,31-Jul-22,RT1,22,30.0,73.33,Standard,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend
6497,18560,31-Jul-22,RT2,34,40.0,85.00,Elite,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend
6498,18560,31-Jul-22,RT3,17,24.0,70.83,Premium,Atliq City,Business,Hyderabad,Jul 22,W 32,weekend


In [187]:
df_bookings_city.head()

,property_id,check_in_date,room_category,successful_bookings,capacity,occ %,room_class,property_name,category,city,mmm yy,week no,day_type
0,16559,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
1,16559,10-May-22,RT2,25,41.0,60.98,Elite,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
2,16559,10-May-22,RT3,20,32.0,62.50,Premium,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
3,16559,10-May-22,RT4,13,18.0,72.22,Presidential,Atliq Exotica,Luxury,Mumbai,May 22,W 20,weekeday
4,19562,10-May-22,RT1,18,30.0,60.00,Standard,Atliq Bay,Luxury,Bangalore,May 22,W 20,weekeday


In [188]:
df_bookings_city.shape

(6500, 13)

In [189]:
df_aug22_data.shape

(7, 13)

In [195]:
df_bookings_all = pd.concat([df_bookings_city, df_aug22_data], ignore_index=True, axis=0)

df_bookings_all.tail(3)

,property_id,check_in_date,room_category,successful_bookings,capacity,occ %,room_class,property_name,category,city,mmm yy,week no,day_type,occ%
6504,19560,01-Aug-22,RT1,20,26.0,NaN,Standard,Atliq City,Business,Bangalore,Aug-22,W 32,weekeday,76.92
6505,17561,01-Aug-22,RT1,18,26.0,NaN,Standard,Atliq Blu,Luxury,Mumbai,Aug-22,W 32,weekeday,69.23
6506,17564,01-Aug-22,RT1,10,16.0,NaN,Standard,Atliq Seasons,Business,Mumbai,Aug-22,W 32,weekeday,62.50


#### 6. Print revenue realized per city

In [201]:
df_bookings.head(2)

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
1,May012216558RT12,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640
4,May012216558RT15,16558,27-04-22,1/5/2022,2/5/2022,4.0,RT1,direct online,5.0,Checked Out,10920,10920


In [204]:
df_hotels.head(2)

,property_id,property_name,category,city
0,16558,Atliq Grands,Luxury,Delhi
1,16559,Atliq Exotica,Luxury,Mumbai


In [237]:
df_fact_bookings_city = pd.merge(df_bookings, df_hotels, on='property_id')

df_fact_bookings_city.head(10
                          )

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized,property_name,category,city
0,May012216558RT12,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640,Atliq Grands,Luxury,Delhi
1,May012216558RT15,16558,27-04-22,1/5/2022,2/5/2022,4.0,RT1,direct online,5.0,Checked Out,10920,10920,Atliq Grands,Luxury,Delhi
2,May012216558RT16,16558,1/5/2022,1/5/2022,3/5/2022,2.0,RT1,others,4.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi
3,May012216558RT17,16558,28-04-22,1/5/2022,6/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640,Atliq Grands,Luxury,Delhi
4,May012216558RT18,16558,26-04-22,1/5/2022,3/5/2022,2.0,RT1,logtrip,NaN,No Show,9100,9100,Atliq Grands,Luxury,Delhi
5,May012216558RT19,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,tripster,NaN,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi
6,May012216558RT110,16558,28-04-22,1/5/2022,2/5/2022,1.0,RT1,others,4.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi
7,May012216558RT111,16558,29-04-22,1/5/2022,7/5/2022,1.0,RT1,makeyourtrip,5.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi
8,May012216558RT112,16558,26-04-22,1/5/2022,2/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi
9,May012216558RT113,16558,26-04-22,1/5/2022,7/5/2022,2.0,RT1,makeyourtrip,5.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi


In [214]:
(df_fact_bookings_city.groupby('city')['revenue_realized'].sum()/1000000).round(2).sort_values(ascending=False).astype(str) + 'MN'

city
Mumbai       668.57MN
Bangalore    420.38MN
Hyderabad    325.18MN
Delhi         294.4MN
Name: revenue_realized, dtype: object

***Despite having the highest occupancy in Delhi, the realized revenue is surprisingly low. It's crucial to delve deeper and find out the specific reasons behind this revenue disparity in the capital city.***

#### 7. Print month by month revenue

In [216]:
df_fact_bookings = pd.merge(df_fact_bookings_city, df_date, left_on='check_in_date', right_on='date')
df_fact_bookings.head(2)

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized,property_name,category,city,date,mmm yy,week no,day_type


In [217]:
# No records are merged, we need to check the condition on which we are merging

In [239]:
df_fact_bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   booking_id         0 non-null      object        
 1   property_id        0 non-null      int64         
 2   booking_date       0 non-null      object        
 3   check_in_date      0 non-null      datetime64[ns]
 4   checkout_date      0 non-null      object        
 5   no_guests          0 non-null      float64       
 6   room_category      0 non-null      object        
 7   booking_platform   0 non-null      object        
 8   ratings_given      0 non-null      float64       
 9   booking_status     0 non-null      object        
 10  revenue_generated  0 non-null      int64         
 11  revenue_realized   0 non-null      int64         
 12  property_name      0 non-null      object        
 13  category           0 non-null      object        
 14  city               0 n

In [222]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      92 non-null     object
 1   mmm yy    92 non-null     object
 2   week no   92 non-null     object
 3   day_type  92 non-null     object
dtypes: object(4)
memory usage: 3.0+ KB


In [224]:
# We notice that both condition columns are of object type, we need to conver them to DATETIME datatype using pd.todatetime method

In [251]:
df_fact_bookings_city['check_in_date'] = pd.to_datetime(df_fact_bookings_city['check_in_date'], infer_datetime_format=True, errors='coerce')


df_fact_bookings_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134573 entries, 0 to 134572
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   booking_id         134573 non-null  object        
 1   property_id        134573 non-null  int64         
 2   booking_date       134573 non-null  object        
 3   check_in_date      55790 non-null   datetime64[ns]
 4   checkout_date      134573 non-null  object        
 5   no_guests          134573 non-null  float64       
 6   room_category      134573 non-null  object        
 7   booking_platform   134573 non-null  object        
 8   ratings_given      56676 non-null   float64       
 9   booking_status     134573 non-null  object        
 10  revenue_generated  134573 non-null  int64         
 11  revenue_realized   134573 non-null  int64         
 12  property_name      134573 non-null  object        
 13  category           134573 non-null  object  

C:\Users\Admin\AppData\Local\Temp\ipykernel_3572\3916352188.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_fact_bookings_city['check_in_date'] = pd.to_datetime(df_fact_bookings_city['check_in_date'], infer_datetime_format=True, errors='coerce')


In [253]:
df_date['date']= pd.to_datetime(df_date['date'])

df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      92 non-null     datetime64[ns]
 1   mmm yy    92 non-null     object        
 2   week no   92 non-null     object        
 3   day_type  92 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ KB


In [254]:
# merging both the tables

df_fact_bookings = pd.merge(df_fact_bookings_city, df_date, left_on='check_in_date', right_on  = 'date')



In [255]:
df_fact_bookings.head()

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized,property_name,category,city,date,mmm yy,week no,day_type
0,May052216558RT11,16558,15-04-22,2022-05-05,7/5/2022,3.0,RT1,tripster,5.0,Checked Out,10010,10010,Atliq Grands,Luxury,Delhi,2022-05-05,May 22,W 19,weekeday
1,May052216558RT12,16558,30-04-22,2022-05-05,7/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640,Atliq Grands,Luxury,Delhi,2022-05-05,May 22,W 19,weekeday
2,May052216558RT13,16558,1/5/2022,2022-05-05,6/5/2022,3.0,RT1,direct offline,5.0,Checked Out,10010,10010,Atliq Grands,Luxury,Delhi,2022-05-05,May 22,W 19,weekeday
3,May052216558RT14,16558,3/5/2022,2022-05-05,6/5/2022,2.0,RT1,tripster,3.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi,2022-05-05,May 22,W 19,weekeday
4,May052216558RT15,16558,30-04-22,2022-05-05,10/5/2022,4.0,RT1,others,4.0,Checked Out,10920,10920,Atliq Grands,Luxury,Delhi,2022-05-05,May 22,W 19,weekeday


In [259]:
(df_fact_bookings.groupby('mmm yy')['revenue_realized'].sum()/1000000).round(2).astype(str) + ' Mn'


mmm yy
Jul 22    60.28 Mn
Jun 22     52.9 Mn
May 22    60.96 Mn
Name: revenue_realized, dtype: object

### 8. Print revenue realized per hotel type

In [262]:
df_fact_bookings['property_name'].unique()

array(['Atliq Grands', 'Atliq Exotica', 'Atliq City', 'Atliq Blu',
       'Atliq Bay', 'Atliq Palace', 'Atliq Seasons'], dtype=object)

In [265]:
(df_fact_bookings.groupby('property_name')['revenue_realized'].sum()/1000000).round(2).astype(str) + ' Mn'

property_name
Atliq Bay        26.94 Mn
Atliq Blu        26.46 Mn
Atliq City       29.05 Mn
Atliq Exotica    32.44 Mn
Atliq Grands     21.64 Mn
Atliq Palace     30.95 Mn
Atliq Seasons     6.67 Mn
Name: revenue_realized, dtype: object

### 9.  Print average rating per city

In [266]:
df_fact_bookings.groupby('city')['ratings_given'].mean()

city
Bangalore    3.410464
Delhi        3.785784
Hyderabad    3.653743
Mumbai       3.629671
Name: ratings_given, dtype: float64

### 10. Print a pie chart of revenue realized per booking platform

In [270]:
(df_fact_bookings.groupby('booking_platform')['revenue_realized'].sum()/1000000).round(2).sort_values(ascending=False).astype(str) + ' Mn'

booking_platform
others            72.31 Mn
makeyourtrip      34.03 Mn
logtrip           18.61 Mn
direct online     17.49 Mn
tripster          11.96 Mn
journey           10.76 Mn
direct offline     8.99 Mn
Name: revenue_realized, dtype: object